- Reference: https://www.cnblogs.com/harrylyx/p/15141986.html

# pip install 非默认包

In [4]:
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pyarrow
! pip install -i https://pypi.tuna.tsinghua.edu.cn/simple fastparquet

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
import random
import string
import pickle

import numpy as np
import pandas as pd

import tables

import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq

import fastparquet as fp

# 生成随机数据

- Reference: http://t.zoukankan.com/harrylyx-p-15141986.html

In [6]:
# 变量定义
row_num = int(1e7)
col_num = 5
str_len = 4
str_nunique = 10 # 字符串组合数量

# 生成随机整数构成的DataFrame
int_matrix = np.random.randint(0, 100, size=(row_num, col_num))
df = pd.DataFrame(int_matrix, columns=['int_%d' % i for i in range(col_num)])

# 生成随机小数构成的DataFrame
float_matrix = np.random.rand(row_num, col_num)
df = pd.concat(
    (df, pd.DataFrame(float_matrix, columns=['float_%d' % i for i in range(col_num)])), axis=1)

# 生成随机字符串构成的DataFrame
str_list = [''.join(random.sample(string.ascii_letters, str_len)) for _ in range(str_nunique)]
for i in range(col_num):
    sr = pd.Series(str_list*(row_num//str_nunique)).sample(frac=1, random_state=i)
    df['str_%d' % i] = sr

# 合并在一起的DataFrame的大小
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 15 columns):
 #   Column   Dtype  
---  ------   -----  
 0   int_0    int32  
 1   int_1    int32  
 2   int_2    int32  
 3   int_3    int32  
 4   int_4    int32  
 5   float_0  float64
 6   float_1  float64
 7   float_2  float64
 8   float_3  float64
 9   float_4  float64
 10  str_0    object 
 11  str_1    object 
 12  str_2    object 
 13  str_3    object 
 14  str_4    object 
dtypes: float64(5), int32(5), object(5)
memory usage: 953.7+ MB
None


# 定义file_format_dict

In [7]:
file_format_dict = {
    'csv':           'df_csv.csv', 
    'csv+gz':        'df_csv.csv.gz', 
    'csv+zip':       'df_csv.csv.zip', 
    'pkl':           'df_pkl.pkl', 
    'npy':           'df_npy.npy', 
    'hdf':           'df_hdf.h5', 
    'hdf+zlib+lvl9': 'df_hdf_comp_lvl9.h5',
    'parquet':       'df_parquet.parquet', 
    'feather':       'df_feather.feather'
}

print(file_format_dict.keys())
print(file_format_dict.values())

dict_keys(['csv', 'csv+gz', 'csv+zip', 'pkl', 'npy', 'hdf', 'hdf+zlib+lvl9', 'parquet', 'feather'])
dict_values(['df_csv.csv', 'df_csv.csv.gz', 'df_csv.csv.zip', 'df_pkl.pkl', 'df_npy.npy', 'df_hdf.h5', 'df_hdf_comp_lvl9.h5', 'df_parquet.parquet', 'df_feather.feather'])


# 保存文件

## csv

In [8]:
# 写入
f_name = file_format_dict['csv']
%time df.to_csv(f_name, index=False)

Wall time: 1min 1s


## csv+gz

In [9]:
# 写入
f_name = file_format_dict['csv+gz']
%time df.to_csv(f_name, index=False, compression='gzip')

Wall time: 3min 3s


## csv+zip

In [10]:
# 写入
f_name = file_format_dict['csv+zip']
%time df.to_csv(f_name, index=False, compression='zip')

Wall time: 2min 9s


## pkl

In [11]:
# 写入
f_name = file_format_dict['pkl']
%time  with open(f_name, 'wb') as f: pickle.dump(df, f)

Wall time: 2.17 s


## npy

In [12]:
# 写入
f_name = file_format_dict['npy']
%time with open(f_name, "wb") as f:   np.save(f, arr=df.values)

Wall time: 7.46 s


## hdf

In [13]:
# 写入
f_name = file_format_dict['hdf']
%time df.to_hdf(f_name, key='df')

Wall time: 2.94 s


## hdf+zlib+level9

In [14]:
# 写入
f_name = file_format_dict['hdf+zlib+lvl9']
%time df.to_hdf(f_name, complevel=9, key='df')

Wall time: 11.3 s


## parquet

In [15]:
# 写入
f_name = file_format_dict['parquet']
%time pq.write_table(pa.Table.from_pandas(df), f_name)

Wall time: 5.17 s


## feather

In [16]:
# 写入
f_name = file_format_dict['feather']
%time feather.write_feather(df, f_name)

Wall time: 2.38 s


# 读取文件

## csv

In [17]:
# 读取
f_name = file_format_dict['csv']
%time df = pd.read_csv(f_name)

Wall time: 10.1 s


## csv+gv

In [18]:
# 读取
f_name = file_format_dict['csv+gz']
%time df = pd.read_csv(f_name, compression='gzip')

Wall time: 15.5 s


## csv+zip

In [19]:
# 读取
f_name = file_format_dict['csv+zip']
%time df = pd.read_csv(f_name, compression='zip')

Wall time: 14.4 s


## pkl

In [20]:
# 读取
f_name = file_format_dict['pkl']
%time with open(f_name, 'rb') as f:   df = pickle.load(f)

Wall time: 2.5 s


## npy

In [21]:
# 读取
f_name = file_format_dict['npy']
%time with open(f_name, "rb") as f: df_array = np.load(f, allow_pickle=True);   df = pd.DataFrame(df_array)

Wall time: 9.47 s


## hdf

In [22]:
# 读取
f_name = file_format_dict['hdf']
%time df = pd.read_hdf(f_name, key='df')

Wall time: 7.35 s


## hdf+zlib+level9

In [23]:
# 读取
f_name = file_format_dict['hdf+zlib+lvl9']
%time df = pd.read_hdf(f_name, key='df')

Wall time: 4.87 s


## parquet

In [24]:
# 读取
f_name = file_format_dict['parquet']
%time df = pq.read_table(f_name).to_pandas()

Wall time: 3.74 s


## feather

In [25]:
# 读取
f_name = file_format_dict['feather']
%time df = feather.read_feather(f_name)

Wall time: 1.82 s


# Compare

## 获取file size

In [26]:
import os

def get_file_mb_size(path):
    byte_size = os.path.getsize(path)
    mb_size = byte_size/(1024*1024)
    return '{}：{}MB'.format(path, round(mb_size, 2))

file_sizes = [get_file_mb_size(f) for f in file_format_dict.values()]
file_sizes

['df_csv.csv：1305.1MB',
 'df_csv.csv.gz：517.82MB',
 'df_csv.csv.zip：520.17MB',
 'df_pkl.pkl：667.67MB',
 'df_npy.npy：620.17MB',
 'df_hdf.h5：744.99MB',
 'df_hdf_comp_lvl9.h5：459.22MB',
 'df_parquet.parquet：425.82MB',
 'df_feather.feather：661.36MB']

## 获取Wall Times，分成save_t和read_t

In [37]:
outputs = ''

In [38]:
%%javascript

// 获取当前页面上的Wall Times
//通过class获取标签元素,得到的是包含对象的数组 (IE8+ 支持)
var elems = document.querySelectorAll('div.output_subarea > pre');
var elems_text = new Array();
for (var i = 0; i < elems.length; i++) {
    var txt = elems[i].textContent;
    if(txt.indexOf('Wall time')!=-1) {
        elems_text.push(txt);
    }
}
var output_texts = elems_text.join().replace(/[\r\n]/g,"");
// console.log(output_texts);

var kernel = IPython.notebook.kernel;
var command = "outputs='" + output_texts + "'";
console.log(command);
kernel.execute(command);

<IPython.core.display.Javascript object>

In [39]:
times = outputs.replace(',', '').split('Wall time: ')[1:]
mid = len(times)//2
save_ts, read_ts = times[:mid], times[mid:]
print('写入时间', save_ts)
print('读取时间', read_ts)

写入时间 ['1min 1s', '3min 3s', '2min 9s', '2.17 s', '7.46 s', '2.94 s', '11.3 s', '5.17 s', '2.38 s']
读取时间 ['10.1 s', '15.5 s', '14.4 s', '2.5 s', '9.47 s', '7.35 s', '4.87 s', '3.74 s', '1.82 s']


## 生成res_df

In [44]:
#        文件类型  | 文件名 | 写入时间(s) | 读取时间(s)  | 存储空间(MB)
res_df = pd.DataFrame([file_format_dict.keys(),
                       file_format_dict.values(),
                       save_ts,
                       read_ts,
                       [get_file_mb_size(f).split('：')[-1] for f in file_format_dict.values()]
                      ]).T
res_df.columns = ['f_format', 'f_name', 'save_t', 'read_t', 'f_size']

res_df['save_t(s)'] = res_df['save_t'].apply(lambda x: x.replace('s', ''))\
                                        .apply(lambda x: 60*float(x.split('min')[0])+float(x.split('min')[1]) if ('min' in x) else float(x))
res_df['read_t(s)'] = res_df['read_t'].apply(lambda x: x.replace('s', '')).astype(float)
res_df['f_size(MB)'] = res_df['f_size'].apply(lambda x: x[:-2]).astype(float)

res_df

,f_format,f_name,save_t,read_t,f_size,save_t(s),read_t(s),f_size(MB)
0,csv,df_csv.csv,1min 1s,10.1 s,1305.1MB,61.00,10.10,1305.10
1,csv+gz,df_csv.csv.gz,3min 3s,15.5 s,517.82MB,183.00,15.50,517.82
2,csv+zip,df_csv.csv.zip,2min 9s,14.4 s,520.17MB,129.00,14.40,520.17
3,pkl,df_pkl.pkl,2.17 s,2.5 s,667.67MB,2.17,2.50,667.67
4,npy,df_npy.npy,7.46 s,9.47 s,620.17MB,7.46,9.47,620.17
5,hdf,df_hdf.h5,2.94 s,7.35 s,744.99MB,2.94,7.35,744.99
6,hdf+zlib+lvl9,df_hdf_comp_lvl9.h5,11.3 s,4.87 s,459.22MB,11.30,4.87,459.22
7,parquet,df_parquet.parquet,5.17 s,3.74 s,425.82MB,5.17,3.74,425.82
8,feather,df_feather.feather,2.38 s,1.82 s,661.36MB,2.38,1.82,661.36


## 不同评价指标下降序排列（Pyecharts）

In [41]:
from pyecharts import options as opts
from pyecharts.charts import Tab
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts

In [49]:
def gen_pyechart_table(data_df, col_name=None):
    show_df = data_df.copy()
    if col_name:
        show_df['{} rank'.format(col_name)] = show_df[col_name].rank().astype(int)
        show_df.sort_values('{} rank'.format(col_name), ascending=True, inplace=True)
    
    table = Table()
    headers = list(show_df.columns)
    rows = [list(row) for i, row in show_df.iterrows()]
    table.add(headers, rows)
    table.set_global_opts(
        title_opts=ComponentTitleOpts(title="Top100")
    )
    return table

tab = Tab()
tab.add(gen_pyechart_table(res_df), "Origin")
tab.add(gen_pyechart_table(res_df, 'save_t(s)'), "save_t降序")
tab.add(gen_pyechart_table(res_df, 'read_t(s)'), "read_t降序")
tab.add(gen_pyechart_table(res_df, 'f_size(MB)'), "f_size降序")
tab.render_notebook()

# 删除全部数据文件

In [90]:
for v in file_format_dict.values():
    print('{} Delete~'.format(v))
    os.remove(v)
print('Done~')

df_csv.csv Delete~
df_csv.csv.gz Delete~
df_csv.csv.zip Delete~
df_pkl.pkl Delete~
df_npy.npy Delete~
df_hdf.h5 Delete~
df_hdf_comp_lvl9.h5 Delete~
df_parquet.parquet Delete~
df_feather.feather Delete~
Done~
